In [20]:
import os
import re
import json
import requests
import urllib.parse
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from openai import OpenAI


def main():
   
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    with open("script.txt", "r", encoding="utf-8") as file: 
        script = file.read()

    while True:
        
        user_input = input("Enter something (:q to quit): ")
        
        if(user_input == ":q"):
            print("Goodbye!")
            break
        
        system_msg = {
            "role": "system",
            "content": script
        }
        
        chat_completion = client.chat.completions.create(
            messages=[
                system_msg,
                {
                    "role": "user",
                    "content": user_input,
                }
            ],
            model="gpt-4o",
        )        
        response_text = chat_completion.choices[0].message.content
        
        print(f"AI answer : {response_text}")

        # case 1 : Recommend a game genre to the user
        findGenreList = find_genre(response_text)       
        if(findGenreList): find_discounted_games_by_genre(findGenreList)

        # case 2 : Recommend newly released games to the user
        else :
            if(get_topic(response_text) == "신규 게임"):
                get_newly_games()
    
def get_topic(response_text):

    return re.findall(r'\{(.*?)\}', response_text)[0]
    
def find_genre(gameGenreStr):

    gameGenreList = [ "캐주얼", "액션", "어드벤처", "시뮬레이션", "분위기 있는", "RPG", 
        "전략", "핵 앤 슬래시", "릴랙싱", "레이싱", "오픈 월드", "슈팅", 
        "격투", "리듬", "비주얼 노벨", "퍼즐", "풍부한 스토리", "건설", 
        "생활 시뮬레이션", "연애", "우주", "군사", "타워 디펜스", "스포츠"
    ]
    
    findGenre = re.findall(r'\[(.*?)\]', gameGenreStr)
    result = list(filter(lambda words: words in gameGenreList, findGenre))
    
    return result


def get_gameinfo_by_html(html, typeDic):

    soup = bs(html, "html.parser")
    
    gameInfoList = []
      
    gameBlocks = soup.find_all(typeDic["block"]["elem"], class_=typeDic["block"]["attr"])

    for block in gameBlocks:

        gameName = block.find(typeDic["name"]["elem"], class_=typeDic["name"]["attr"]).text
        gameDiscountRate = block.find("div", class_="discount_pct").text if block.find("div", class_="discount_pct") else "0%"
        gameOriginalPrice = block.find("div", class_="discount_original_price").text if block.find("div", class_="discount_original_price") else "0"
        gameFinalPrice = block.find("div", class_="discount_final_price").text
                
        gameInfoList.append((gameName, gameDiscountRate, gameOriginalPrice, gameFinalPrice))           

    return gameInfoList
            
            
def find_discounted_games_by_genre(gameGenre):

    with open("steamGenreTags.txt", "r", encoding="utf-8") as file:
        steamGenreTag = json.load(file)
    
    typeDic = {
        "block" : {"elem":"a", "attr":"search_result_row"},
        "name"  : {"elem":"span", "attr":"title"},
        "img"   : {"elem":"div", "attr":"search_capsule"}
    }

    url = "https://store.steampowered.com/search/?l=koreana&specials=1&tags="
    for genre in gameGenre:
        if (genre in steamGenreTag): url += ("%2C" + steamGenreTag[genre])
        
    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.page_source

    gameInfoList = get_gameinfo_by_html(html, typeDic)

    for elem in gameInfoList:
        print(f"game name: {elem[0]}, Discount rate: {elem[1]}, Original price: {elem[2]}, Final price: {elem[3]}") 

def get_newly_games():
    
    typeDic = {
        "block" : {"elem":"a", "attr":"tab_item"},
        "name"  : {"elem":"div", "attr":"tab_item_name"},
        "img"   : {"elem":"div", "attr":"tab_item_cap"}
    }

    url = "https://store.steampowered.com/explore/new/?l=koreana"

    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.page_source

    gameInfoList = get_gameinfo_by_html(html, typeDic)

    for elem in gameInfoList:
        print(f"game name: {elem[0]}, Discount rate: {elem[1]}, Original price: {elem[2]}, Final price: {elem[3]}")    

if __name__ == "__main__":
    main()

Enter something (:q to quit):  새로운 게임을 추천해줘


AI answer : {신규 게임}
game name: On Your Tail, Discount rate: -10%, Original price: ₩ 37,500, Final price: ₩ 33,750
game name: The Cabin Factory, Discount rate: -10%, Original price: ₩ 3,400, Final price: ₩ 3,060
game name: Murky Divers, Discount rate: -25%, Original price: ₩ 10,100, Final price: ₩ 7,570
game name: MiSide, Discount rate: -10%, Original price: ₩ 16,500, Final price: ₩ 14,850
game name: Ballionaire, Discount rate: -19%, Original price: ₩ 12,340, Final price: ₩ 9,990
game name: 인디아나 존스: 그레이트 서클 , Discount rate: 0%, Original price: 0, Final price: ₩ 79,990
game name: 마블 라이벌즈, Discount rate: 0%, Original price: 0, Final price: 무료
game name: Get To Work, Discount rate: 0%, Original price: 0, Final price: ₩ 14,500
game name: Void Crew, Discount rate: 0%, Original price: 0, Final price: ₩ 29,000
game name: Strinova, Discount rate: 0%, Original price: 0, Final price: 무료
game name: S.T.A.L.K.E.R. 2: Heart of Chornobyl, Discount rate: 0%, Original price: 0, Final price: ₩ 64,900
ga

Enter something (:q to quit):  :q


Goodbye!
